In [2]:
import pandas as pd
import bs4
import os

In [3]:
data = pd.read_csv('data/laurier.csv')

In [4]:
data['content'] = data['raw'].apply(lambda x: str(bs4.BeautifulSoup(x, 'html.parser').select_one('div.content')))

In [5]:
def getCourseData(url: str, content: str):
    soup = bs4.BeautifulSoup(content, 'html.parser')
    
    header = soup.select_one('div.content > h1').contents

    if len(header) < 2:
        return {
            'url' : url,
            'title': None,
            'name': None,
            'credits': None,
            'description': None
        }

    title = header[0].text.strip()
    name = header[2].text.strip()
    credits = header[4].text.strip()
    description = soup.select_one('div.content > p')

    return {
        'url' : url,
        'title': title,
        'name': name,
        'credits': credits,
        'description': description.text.strip() if description else None
    }

In [6]:
data.head()

,url,title,timestamp,raw,content
0,https://academic-calendar.wlu.ca/course.php?c=...,Wilfrid Laurier University - Undergraduate Aca...,2023-12-05 00:41:33.668857,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<div class=""content"">\n<h1>MU137<br/><span sty..."
1,https://academic-calendar.wlu.ca/course.php?c=...,Wilfrid Laurier University - Undergraduate Aca...,2023-12-05 00:41:33.928843,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<div class=""content"">\n<h1>MU360<br/><span sty..."
2,https://academic-calendar.wlu.ca/course.php?c=...,Wilfrid Laurier University - Undergraduate Aca...,2023-12-05 00:41:34.137938,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<div class=""content"">\n<h1>MU252<br/><span sty..."
3,https://academic-calendar.wlu.ca/course.php?c=...,Wilfrid Laurier University - Undergraduate Aca...,2023-12-05 00:41:34.333075,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<div class=""content"">\n<h1>MU162<br/><span sty..."
4,https://academic-calendar.wlu.ca/course.php?c=...,Wilfrid Laurier University - Undergraduate Aca...,2023-12-05 00:41:34.538969,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...","<div class=""content"">\n<h1>MU236<br/><span sty..."


In [7]:
content = []
for row in data[data['url'].str.startswith('https://academic-calendar.wlu.ca/course.php')].itertuples():
    content.append(getCourseData(row.url, row.content))

courses = pd.DataFrame(content).dropna()

In [8]:
courses.sort_values('title')

,url,title,name,credits,description
1281,https://academic-calendar.wlu.ca/course.php?c=...,AB101,Elementary Modern Standard Arabic I,0.5 Credit,Taught in Arabic. This course will introduce s...
3892,https://academic-calendar.wlu.ca/course.php?c=...,AB101,Elementary Modern Standard Arabic I,0.5 Credit,Taught in Arabic. This course will introduce s...
1290,https://academic-calendar.wlu.ca/course.php?c=...,AB102,Elementary Modern Standard Arabic II,0.5 Credit,Taught in Arabic. A continuation of AB101. In ...
3901,https://academic-calendar.wlu.ca/course.php?c=...,AB102,Elementary Modern Standard Arabic II,0.5 Credit,Taught in Arabic. A continuation of AB101. In ...
3906,https://academic-calendar.wlu.ca/course.php?c=...,AB201,Intermediate Arabic I,0.5 Credit,Taught in Arabic. A practical course which rev...
...,...,...,...,...,...
4291,https://academic-calendar.wlu.ca/course.php?c=...,YC430/SOJE430,Youth Cultures,0.5 Credit,This course engages students in critical and c...
1115,https://academic-calendar.wlu.ca/course.php?c=...,YC440,Youth and Sexuality,0.5 Credit,This course examines youth sexuality from an i...
3734,https://academic-calendar.wlu.ca/course.php?c=...,YC440,Youth and Sexuality,0.5 Credit,This course examines youth sexuality from an i...
1697,https://academic-calendar.wlu.ca/course.php?c=...,YC490,Directed Research,0.5 Credit,Individual study on a specialized topic. Stude...


In [2]:
import os
import pandas as pd

In [12]:
DATA_DIR = os.path.join('app', 'data')
df = pd.read_csv(os.path.join(DATA_DIR, 'laurier_courses.csv')).drop_duplicates('title')
# add 'laurier' as multindex
df['school'] = 'laurier'
df = df.set_index(['school', 'title'])

In [18]:
df.to_csv('app/data/laurier_courses.csv')

In [23]:
df2 = pd.read_csv('app/data/laurier_courses.csv', index_col=['school', 'title'])